In [2]:
%pip install -U transformers==4.32.0 \
             datasets==2.14.4 \
             diffusers==0.20.0 \
             accelerate==0.21.0 \
             torch==2.0.1 \
             torchvision==0.15.2 \
             sentencepiece==0.1.99

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Obtaining dependency information for transformers==4.32.0 from https://files.pythonhosted.org/packages/ae/95/283a1c004430bd2a9425d6937fc545dd49a4e4592feb76be0299a14e2378/transformers-4.32.0-py3-none-any.whl.metadata
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 54.6 MB/s eta 0:00:00
  Obtaining dependency information for datasets==2.14.4 from https://files.pythonhosted.org/packages/66/f8/38298237d18d4b6a8ee5dfe390e97bed5adb8e01ec6f9680c0ddf3066728/datasets-2.14.4-py3-none-any.whl.metadata
  Obtaining dependency information for diffusers==0.20.0 from https://files.pythonhosted.org/packages/24/4a/b45557513dfcfb6d69e5c138325a0d7431943792b662e482dcd886c11bb4/diffusers-0.20.0-py3-none-any.whl.metadata
  Obtaining dependency information for accelerate==0.21.0 from https://files.pythonhosted.org/packages/70/f9/c381bcdd0c3829d723aa14eec8e75c6c377b4ca61ec68b8093d9f35fc7a7/accelerate-0.21.0-py3-none-any.whl.m

# TODO:  Read and summarize this: 

https://huggingface.co/docs/transformers/main/en/model_doc/idefics#transformers.IdeficsImageProcessor

and

https://arxiv.org/pdf/2306.16527.pdf


## Highlights
Based on Flamingo, trained on open dataset.

## Dataset
We introduce the OBELICS dataset, an open web-scale filtered dataset of interleaved image-text documents comprising 141 million web pages extracted from Common Crawl, 353 million associated images, and 115 billion text tokens. We describe the dataset creation process, present comprehensive filtering rules, and provide an analysis of the dataset’s content.

## Model 
We train an 80 billion parameters vision and language model on the dataset and obtain competitive performance on various multimodal benchmarks. We release the code to reproduce the dataset along with the dataset itself.

In [3]:
import torch
from transformers import IdeficsForVisionText2Text, AutoProcessor

device = "cuda" if torch.cuda.is_available() else "cpu"

model_name = "HuggingFaceM4/idefics-9b-instruct"
model = IdeficsForVisionText2Text.from_pretrained(model_name, torch_dtype=torch.bfloat16).to(device)
processor = AutoProcessor.from_pretrained(model_name)

# Generation args
exit_condition = processor.tokenizer("<end_of_utterance>", add_special_tokens=False).input_ids
bad_words_ids = processor.tokenizer(["<image>", "<fake_token_around_image>"], add_special_tokens=False).input_ids

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

# Zero-shot inference

![](https://hips.hearstapps.com/hmg-prod/images/dog-puns-1581708208.jpg)

In [69]:
url = "https://hips.hearstapps.com/hmg-prod/images/dog-puns-1581708208.jpg"
img = processor.image_processor.fetch_images([url])[0]

prompts = [
    "\nUser:",
    img,
    "Describe this image.\nAssistant: ",
]

inputs = processor(prompts, return_tensors="pt", debug=True).to(device)

generated_ids = model.generate(**inputs, eos_token_id=exit_condition, bad_words_ids=bad_words_ids, max_length=100)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
generated_text

full_text='<s>\nUser:<fake_token_around_image><image><fake_token_around_image>Describe this image.\nAssistant:'


"<s> \nUser:<fake_token_around_image><image><fake_token_around_image> Describe this image.\nAssistant: The image features a small dog wearing a pair of black glasses, giving it a unique and adorable appearance. The dog is positioned in the center of the frame, and its glasses cover a significant portion of its face. The dog appears to be looking directly at the camera, capturing the viewer's attention. The background is relatively simple, with no other objects or elements distract"

# One-shot Inference to guide the description using a complete example

![](https://hips.hearstapps.com/hmg-prod/images/cute-photos-of-cats-in-grass-1593184777.jpg)
![](https://hips.hearstapps.com/hmg-prod/images/dog-puns-1581708208.jpg)

In [72]:
url = "https://hips.hearstapps.com/hmg-prod/images/cute-photos-of-cats-in-grass-1593184777.jpg"
img = processor.image_processor.fetch_images([url])[0]

# Either use img or url
prompts = [
    "User:",
    img,
    "Describe this image."
    "Assistant: An image of two kittens in grass."
    "User:",
    "https://hips.hearstapps.com/hmg-prod/images/dog-puns-1581708208.jpg",
    "Describe this image.",
    "Assistant: "
]

inputs = processor(prompts, return_tensors="pt", debug=True).to(device)

generated_ids = model.generate(**inputs, eos_token_id=exit_condition, bad_words_ids=bad_words_ids, max_length=100)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
generated_text

full_text='<s>User:<fake_token_around_image><image><fake_token_around_image>Describe this image.Assistant: An image of two kittens in grass.User:<fake_token_around_image><image><fake_token_around_image>Describe this image.<end_of_utterance>Assistant:'


'User: Describe this image.Assistant: An image of two kittens in grass.User: Describe this image. Assistant: A dog wearing glasses and a black shirt.'

# Show special characters injected around the images

In [73]:
generated_ids = model.generate(**inputs, eos_token_id=exit_condition, bad_words_ids=bad_words_ids, max_length=100)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
generated_text

'<s> User:<fake_token_around_image><image><fake_token_around_image> Describe this image.Assistant: An image of two kittens in grass.User:<fake_token_around_image><image><fake_token_around_image> Describe this image.<end_of_utterance> Assistant: A dog wearing glasses and a black shirt.<end_of_utterance>'

# Ask Questions About Text in the Image

![](https://dsoaws.s3.amazonaws.com/gaia/11_multimodal/img/movie-premiere.png)

In [75]:
url = "https://dsoaws.s3.amazonaws.com/gaia/11_multimodal/img/movie-premiere.png"
img = processor.image_processor.fetch_images([url])[0]

prompts = [
    "User: ",
    img,
    "Describe this image.",
    "Assistant: ",
]

inputs = processor(prompts, return_tensors="pt", debug=True).to(device)

generated_ids = model.generate(**inputs, eos_token_id=exit_condition, bad_words_ids=bad_words_ids, max_length=100)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=False)[0]
generated_text

full_text='<s>User:<fake_token_around_image><image><fake_token_around_image>Describe this image.<end_of_utterance>Assistant:'


'<s> User:<fake_token_around_image><image><fake_token_around_image> Describe this image.<end_of_utterance> Assistant: The image is a movie poster featuring a man and a woman walking down a street.<end_of_utterance>'

In [81]:
url = "https://dsoaws.s3.amazonaws.com/gaia/11_multimodal/img/movie-premiere.png"
img = processor.image_processor.fetch_images([url])[0]

prompts = [
    "User: ",
    img,
    "When does this movie premiere.",
    "Assistant: ",
]

inputs = processor(prompts, return_tensors="pt").to(device)

generated_ids = model.generate(**inputs, max_length=100) # eos_token_id=exit_condition, bad_words_ids=bad_words_ids, max_length=100)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_text)

User: When does this movie premiere. Assistant: The movie premieres on June 24.


![](https://dsoaws.s3.amazonaws.com/gaia/11_multimodal/img/baby-groot.jpg)

In [85]:
url = "https://dsoaws.s3.amazonaws.com/gaia/11_multimodal/img/baby-groot.jpg"
img = processor.image_processor.fetch_images([url])[0]

prompts = [
    "User: ",
    img,
    "Which movie is this character from?",
    "Assistant: ",
]

inputs = processor(prompts, return_tensors="pt").to(device)

generated_ids = model.generate(**inputs, max_length=100) # eos_token_id=exit_condition, bad_words_ids=bad_words_ids)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_text)

User: Which movie is this character from?. Assistant: This character is from the movie "Guardians of the Galaxy Vol. 2."


# Chain of thought

![](https://dsoaws.s3.amazonaws.com/gaia/11_multimodal/img/movie-premiere.png)

In [92]:
url = "https://dsoaws.s3.amazonaws.com/gaia/11_multimodal/img/baby-groot.jpg"
img = processor.image_processor.fetch_images([url])[0]

prompts = [
    "User: ",
    img,
    "Who produced the movie that features this character?",
    "Assistant: ",
]

inputs = processor(prompts, return_tensors="pt").to(device)

generated_ids = model.generate(**inputs, max_length=100) #, eos_token_id=exit_condition, bad_words_ids=bad_words_ids)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_text)

User: Who produced the movie that features this character? Assistant: Marvel Studios produced the movie that features this character.


![](https://dsoaws.s3.amazonaws.com/gaia/11_multimodal/img/margherita-pizza.jpg)

In [9]:
url = "https://dsoaws.s3.amazonaws.com/gaia/11_multimodal/img/margherita-pizza.jpg"
img = processor.image_processor.fetch_images([url])[0]

prompts = [
    "User: ",
    img,
    "What is this and how do I make this?",
    "Assistant: ",
]

inputs = processor(prompts, return_tensors="pt").to(device)

generated_ids = model.generate(**inputs, max_length=1000, eos_token_id=exit_condition, bad_words_ids=bad_words_ids)
generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
print(generated_text)

User: What is this and how do I make this? Assistant: This is a pizza, and to make it, you will need the following ingredients: pizza dough, tomato sauce, mozzarella cheese, and basil leaves.

1. Preheat your oven to the highest temperature it can reach, usually around 500°F (260°C).
2. Roll out your pizza dough on a floured surface to your desired thickness.
3. Spread a layer of tomato sauce on the pizza dough, leaving a small border around the edges for the crust.
4. Sprinkle mozzarella cheese over the tomato sauce.
5. Add your desired toppings, such as basil leaves, pepperoni, or any other preferred ingredients.
6. Place the pizza on a baking sheet or pizza stone and bake in the preheated oven for 10-15 minutes or until the crust is golden brown and the cheese is melted and bubbly.
7. Remove the pizza from the oven and let it cool for a few minutes before slicing and serving.

Enjoy your homemade pizza!
